In [1]:
import time
import datetime

from experiment.utils import dbutils, logger, transformation
from experiment.utils.tables.upload_tasks_table import UploadTasksTable
from experiment.api import label_studio

import openai

In [2]:
db = dbutils.DatabaseUtils()
lg = logger.Logger(
    logging_level="DEBUG",
    log_file=transformation.get_project_root() / "tmp" / "report_prompting.log",
)

In [3]:
PROMPT_N_MORE_REPORTS = 200
PROMPT = "Perform the following transformation on the report: Translate into English" 

In [4]:
reports_raw, Base = UploadTasksTable()

In [5]:
# # run the dbt model to generate tables from scratch
# Base.metadata.create_all(db.engine)
# db.run_dbt_model('upload_tasks')

### 1) Select Reports

In [6]:
# get reports directly from database
query = """
            SELECT * FROM annotation.upload_tasks ut 
            ORDER BY patient_report_count DESC, report_length DESC 
        """

# get values from the database
df_reports = db.read_sql_query(query)
df_reports.head()

,report_id,patient_no,protocol_no,report_original,report_length,report_prompted,patient_report_count
0,5408,2005183286,21302877,RAPOR TARİHİ: 24.06.2021 FİLM NO: 12217711...,79,REPORT DATE: 24.06.2021 FILM NO: 12217711\...,27
1,6125,2006000036,22202602,RAPOR TARİHİ:01/02/2022 TETKİK NO: ...,83,REPORT DATE: 01/02/2022 EXAMINATION...,26
2,16913,2008509284,22344550,RAPOR TARİHİ: 21/02/2022 FİLM NO:12506694\n\...,165,REPORT DATE: 02/21/2022 FILM NUMBER: 1250669...,20
3,16909,2008509284,22344550,RAPOR TARİHİ:08/02/2022 FİLM NO:12506525\n\n...,113,REPORT DATE: 08/02/2022 FILM NO: 12506525\n\...,20
4,26,2004004697,22023602,RAPOR TARİHİ:27/12/2021 FİLM NO:\n\nKontrast...,128,REPORT DATE: 27/12/2021 FILM NO:\n \nPlain b...,19


In [7]:
# get annotated reports 
query = """
            SELECT 
                DISTINCT data ->> 'patient_no' as patient_no
            FROM task
            WHERE is_labeled = TRUE
        """

# get values from the database
annotated_patient_nos = db.read_sql_query(query)["patient_no"].to_list()

In [8]:
# get tasks that have been prompted
query = """
            SELECT 
                report_id
            FROM annotation.upload_tasks
            WHERE report_prompted != '' 
        """

# get values from the database
upload_tasks_prompted = db.read_sql_query(query)["report_id"].to_list()

In [9]:
# use only non-prompted reports & non-annotated patients
df_upload_tasks = (
    df_reports.loc[~df_reports["patient_no"].isin(annotated_patient_nos)]
    .loc[~df_reports["report_id"].isin(upload_tasks_prompted)]
    .head(PROMPT_N_MORE_REPORTS)
)

### 2) Prompt Reports

In [10]:
cols_to_upsert = df_upload_tasks.columns.to_list()
cols_to_upsert.remove("report_id")
data_to_insert = []
for _, row in df_upload_tasks.iterrows():
    try:
        data_to_insert.append(
            {
                "report_id": row["report_id"],
                "patient_no": row["patient_no"],
                "protocol_no": row["protocol_no"],
                "report_original": row["report_original"],
                "report_prompted": transformation.prompt_report(
                    report=row["report_original"], prompt=PROMPT
                ),
                "report_length": row["report_length"],
                "patient_report_count": row["patient_report_count"],
            }
        )
    

        db.upsert_values(reports_raw, data_to_insert, cols_to_upsert, ["report_id"])

        time.sleep(20)
    except openai.error.RateLimitError as rate_error:
        # openai restriction: 3 RPM - 200 RPD
        lg.log(f"Rate limit for: {datetime.datetime.now()}", "WARNING")

lg.log(f"Finished prompting {len(data_to_insert)} reports")

KeyboardInterrupt: 

### 3) Upload Tasks to Label Studio

In [11]:
# get reports directly from database
query = """
            SELECT
                report_id,
                patient_no,
                protocol_no,
                report_original,
                report_prompted as text,
                report_length,
                patient_report_count
            FROM
                annotation.upload_tasks
            WHERE
                report_id NOT IN (
                SELECT
                    (DATA ->> 'report_id')::INT AS report_id
                FROM
                    public.task)
                AND report_prompted != ''
        """

# get values from the database
df_upload_tasks = db.read_sql_query(query)

# output tasks as a csv file
output_path = (
    transformation.get_project_root() / "tmp" / "data" / "upload_tasks.csv"
)
df_upload_tasks.to_csv(output_path, index=False)

In [12]:
# upload tasks to label studio
label_studio.upload_csv_tasks(csv_path=output_path, project_id=7)

Scaling dynos... done, now running web at 1:Basic
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"task_count":177,"annotation_count":0,"prediction_count":0,"duration":0.2810530662536621,"file_upload_ids":[40],"could_be_tasks_list":true,"found_formats":{".csv":1},"data_columns":["report_length","patient_no","text","protocol_no","report_id","report_original","patient_report_count"]}

100  289k  100   287  100  288k    361   364k --:--:-- --:--:-- --:--:--  366k


In [13]:
label_studio.stop_label_studio()

Scaling dynos... done, now running web at 0:Basic
